In [ ]:
import pandas as pd
import nltk
import re
import math
import ast
import numpy as np
from gensim.models import Word2Vec
import gensim
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn import datasets
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import numpy as np
from nltk.stem import WordNetLemmatizer
from keras.layers import LSTM, Dense, Bidirectional, Input,Dropout,BatchNormalization
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from keras.utils import to_categorical
from keras import backend as K
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MinMaxScaler

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Doc2Vec

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/Data/df_train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Data/df_test.csv')

In [ ]:
df_train.head()

,text,label,cleaned_text,embeddings_universal,tokenized,embeddings_doc2vec
0,PACs' Post-Labor Day Spending Favors Kerry (AP...,0,pacs postlabor day spending favors kerry ap ap...,"[0.004684394225478172, -0.07251895219087601, 0...","['pacs', 'postlabor', 'day', 'spending', 'favo...",[ 0.04061217 -0.1222736 -0.41735286 0.091205...
1,Dollar Jumps as the G7 Fades LONDON (Reuters)...,2,dollar jumps g7 fades london reuters dollar su...,"[-0.04605080932378769, -0.06875839084386826, -...","['dollar', 'jumps', 'g7', 'fades', 'london', '...",[-0.08765046 -0.09982859 0.09058072 -0.055263...
2,"Rein in militias, UN tells Sudan THE top UN en...",0,rein militias un tells sudan top un envoy suda...,"[-0.02137177437543869, -0.03128012642264366, 0...","['rein', 'militias', 'un', 'tells', 'sudan', '...",[-0.05815008 -0.08653959 -0.12026519 0.195496...
3,Rooney wins Law praise Ex-Manchester United st...,1,rooney wins law praise exmanchester united str...,"[-0.02186579629778862, -0.06221188232302666, -...","['rooney', 'wins', 'law', 'praise', 'exmanches...",[ 0.22654042 0.29193237 -0.31222776 -0.071419...
4,Viewpoint: School finance deadline unrealistic...,2,viewpoint school finance deadline unrealistic ...,"[0.006163605954498053, -0.06435248255729675, -...","['viewpoint', 'school', 'finance', 'deadline',...",[ 0.33168533 -0.00833228 0.06804036 0.047285...


In [ ]:
X_train = df_train['embeddings_doc2vec'].apply(
    lambda s: np.array(s.strip('[]').split(), dtype=float)
).tolist()
X_train = np.stack(X_train, axis=0)

X_test = df_test['embeddings_doc2vec'].apply(
    lambda s: np.array(s.strip('[]').split(), dtype=float)
).tolist()
X_test = np.stack(X_test, axis=0)
labels = df_train['label'].values
le = LabelEncoder()
le.fit(labels)
y_train = le.transform(labels)
y_test = le.transform(df_test['label'].values)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
print(df_train['embeddings_doc2vec'].head())

0    [ 0.04061217 -0.1222736  -0.41735286  0.091205...
1    [-0.08765046 -0.09982859  0.09058072 -0.055263...
2    [-0.05815008 -0.08653959 -0.12026519  0.195496...
3    [ 0.22654042  0.29193237 -0.31222776 -0.071419...
4    [ 0.33168533 -0.00833228  0.06804036  0.047285...
Name: embeddings_doc2vec, dtype: object


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
rf_classifier = RandomForestClassifier(n_jobs=10,n_estimators=100)
rf_classifier.fit(X_train, y_train)
print("Test Accuracy: ",rf_classifier.score(X_test, y_test))
y_pred=rf_classifier.predict(X_test)
print(classification_report(y_test, y_pred, labels=[0,1,2,3]))

Test Accuracy:  0.6280263157894737
              precision    recall  f1-score   support

           0       0.64      0.63      0.64      1900
           1       0.63      0.75      0.68      1900
           2       0.61      0.49      0.54      1900
           3       0.63      0.64      0.63      1900

    accuracy                           0.63      7600
   macro avg       0.63      0.63      0.62      7600
weighted avg       0.63      0.63      0.62      7600



In [ ]:
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.01, 0.1, 1, 10],
    'solver': ['liblinear', 'lbfgs', 'saga']
}
grid_search = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=5)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_

best_model = grid_search.best_estimator_
accuracy = best_model.score(X_test[:12000], y_test[:12000])
print("Train Accuracy: ", accuracy)
print("Test Accuracy: ",best_model.score(X_test, y_test))
y_pred=best_model.predict(X_test)
print(classification_report(y_test, y_pred, labels=[0,1,2,3]))
print(grid_search.best_estimator_)
grid_search.best_estimator_

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
20 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
    raise ValueError(
ValueError: Solv

Train Accuracy:  0.6836842105263158
Test Accuracy:  0.6836842105263158
              precision    recall  f1-score   support

           0       0.69      0.67      0.68      1900
           1       0.73      0.76      0.74      1900
           2       0.64      0.59      0.62      1900
           3       0.67      0.70      0.69      1900

    accuracy                           0.68      7600
   macro avg       0.68      0.68      0.68      7600
weighted avg       0.68      0.68      0.68      7600

LogisticRegression(C=0.1, max_iter=1000, solver='liblinear')


LogisticRegression(C=0.1, max_iter=1000, solver='liblinear')

In [ ]:
from sklearn.neural_network import MLPClassifier
param_grid = {
    'hidden_layer_sizes': [(256,), (128, 128), (128, 64, 32)],
    'activation': ['relu', 'tanh', 'logistic'],
    'solver': ['adam'],
}

mlp = MLPClassifier(max_iter=1000)
grid_search = GridSearchCV(mlp, param_grid, cv=5)
grid_search.fit(X_train[:12000], y_train[:12000])
best_params = grid_search.best_params_

# Step 3: Model Evaluation
best_model = grid_search.best_estimator_
accuracy = best_model.score(X_test, y_test)
print("Accuracy:", accuracy)
print(best_model)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy: 0.6723684210526316
MLPClassifier(hidden_layer_sizes=(256,), max_iter=1000)


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train2 = scaler.transform(X_train)
scaler = MinMaxScaler()
scaler.fit(X_test)
X_test2 = scaler.transform(X_test)
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 2.0]  # Example values for alpha
}
mnb = MultinomialNB()
grid_search = GridSearchCV(mnb, param_grid, cv=5)
grid_search.fit(X_train2, y_train)
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
accuracy = best_model.score(X_test2, y_test)
print("Accuracy:", accuracy)
print(best_model)

Accuracy: 0.49315789473684213
MultinomialNB(alpha=0.1)


In [ ]:
dt_classifier = DecisionTreeClassifier()
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [40],
    'min_samples_split': [2, 5, 10],
    'max_features': ['auto', 'sqrt', 'log2']
}

grid_search = GridSearchCV(dt_classifier, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train[:12000], y_train[:12000])
best_dt = grid_search.best_estimator_
y_pred = best_dt.predict(X_test)

print("Best Parameters:", grid_search.best_params_)
print("Best Estimator:", best_dt)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/pytho

Best Parameters: {'criterion': 'gini', 'max_depth': 40, 'max_features': 'sqrt', 'min_samples_split': 2}
Best Estimator: DecisionTreeClassifier(max_depth=40, max_features='sqrt')
Accuracy: 0.32776315789473687
Classification Report:
               precision    recall  f1-score   support

           0       0.31      0.29      0.30      1900
           1       0.36      0.36      0.36      1900
           2       0.30      0.30      0.30      1900
           3       0.34      0.36      0.35      1900

    accuracy                           0.33      7600
   macro avg       0.33      0.33      0.33      7600
weighted avg       0.33      0.33      0.33      7600



In [ ]:
print("Test Accuracy: ",best_model.score(X_test2, y_test))
y_pred=best_model.predict(X_test2)
print(classification_report(y_test, y_pred, labels=[0,1,2,3]))

Test Accuracy:  0.49315789473684213
              precision    recall  f1-score   support

           0       0.69      0.47      0.56      1900
           1       0.87      0.29      0.44      1900
           2       0.35      0.88      0.50      1900
           3       0.76      0.33      0.46      1900

    accuracy                           0.49      7600
   macro avg       0.66      0.49      0.49      7600
weighted avg       0.66      0.49      0.49      7600



In [ ]:
estimators = [
    ('mlp', MLPClassifier(alpha=0.1,solver="adam",hidden_layer_sizes=(256,),max_iter=10000)),
    ('mnb', MultinomialNB(alpha=0.1)),
    ('lr', LogisticRegression(C=0.1, max_iter=1000, penalty='l2', solver='liblinear')),
    ('dt', DecisionTreeClassifier(criterion= 'entropy', max_features='auto', min_samples_split=10))
]

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train2 = scaler.transform(X_train)
scaler = MinMaxScaler()
scaler.fit(X_test)
X_test2 = scaler.transform(X_test)
model_voting = VotingClassifier(
    estimators = estimators,
    voting='hard',
    n_jobs=-1,
    flatten_transform=True,
    verbose=1).fit(X_train2, y_train)

print(model_voting.score(X_test2, y_test))
y_pred=model_voting.predict(X_test2)
print(classification_report(y_test, y_pred, labels=[0,1,2,3]))

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


0.6073684210526316
              precision    recall  f1-score   support

           0       0.69      0.64      0.66      1900
           1       0.84      0.49      0.62      1900
           2       0.44      0.85      0.58      1900
           3       0.78      0.45      0.57      1900

    accuracy                           0.61      7600
   macro avg       0.69      0.61      0.61      7600
weighted avg       0.69      0.61      0.61      7600



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf_gbc = GradientBoostingClassifier(n_estimators=10, learning_rate=1.0, random_state=0).fit(X_train, y_train)
print(clf_gbc.score(X_test, y_test))
y_pred=clf_gbc.predict(X_test)
print(classification_report(y_test, y_pred, labels=[0,1,2,3]))

0.5719736842105263
              precision    recall  f1-score   support

           0       0.58      0.58      0.58      1900
           1       0.60      0.66      0.63      1900
           2       0.54      0.49      0.51      1900
           3       0.57      0.56      0.56      1900

    accuracy                           0.57      7600
   macro avg       0.57      0.57      0.57      7600
weighted avg       0.57      0.57      0.57      7600



In [ ]:
clf_gbc = GradientBoostingClassifier(n_estimators=20, learning_rate=1.0, random_state=0).fit(X_train, y_train)
print(clf_gbc.score(X_test, y_test))
y_pred=clf_gbc.predict(X_test)
print(classification_report(y_test, y_pred, labels=[0,1,2,3]))

0.6185526315789474
              precision    recall  f1-score   support

           0       0.62      0.61      0.61      1900
           1       0.65      0.70      0.67      1900
           2       0.60      0.54      0.57      1900
           3       0.60      0.63      0.62      1900

    accuracy                           0.62      7600
   macro avg       0.62      0.62      0.62      7600
weighted avg       0.62      0.62      0.62      7600



In [ ]:
clf_gbc = GradientBoostingClassifier(n_estimators=30, learning_rate=1.0, random_state=0).fit(X_train, y_train)
print(clf_gbc.score(X_test, y_test))
y_pred=clf_gbc.predict(X_test)
print(classification_report(y_test, y_pred, labels=[0,1,2,3]))

0.6301315789473684
              precision    recall  f1-score   support

           0       0.64      0.61      0.62      1900
           1       0.67      0.72      0.69      1900
           2       0.59      0.56      0.58      1900
           3       0.61      0.63      0.62      1900

    accuracy                           0.63      7600
   macro avg       0.63      0.63      0.63      7600
weighted avg       0.63      0.63      0.63      7600



In [ ]:
lr = LogisticRegression(C=0.1, max_iter=10000, penalty='l1', solver='liblinear')
lr.fit(X_train, y_train)
print("Test Accuracy: ",lr.score(X_test, y_test))
y_pred=lr.predict(X_test)
print(classification_report(y_test, y_pred, labels=[0,1,2,3]))

Test Accuracy:  0.6811842105263158
              precision    recall  f1-score   support

           0       0.69      0.67      0.68      1900
           1       0.73      0.75      0.74      1900
           2       0.64      0.60      0.62      1900
           3       0.66      0.70      0.68      1900

    accuracy                           0.68      7600
   macro avg       0.68      0.68      0.68      7600
weighted avg       0.68      0.68      0.68      7600



In [ ]:
mlp = MLPClassifier(alpha=0.1,solver="adam",hidden_layer_sizes=(256,),max_iter=10000).fit(X_train,y_train)
print("Test Accuracy: ",mlp.score(X_test, y_test))
y_pred=mlp.predict(X_test)
print(classification_report(y_test, y_pred, labels=[0,1,2,3]))

Test Accuracy:  0.7556578947368421
              precision    recall  f1-score   support

           0       0.80      0.71      0.75      1900
           1       0.78      0.86      0.82      1900
           2       0.71      0.71      0.71      1900
           3       0.73      0.75      0.74      1900

    accuracy                           0.76      7600
   macro avg       0.76      0.76      0.75      7600
weighted avg       0.76      0.76      0.75      7600

